In [1]:
import os
# change the current working directory
os.chdir('..')

# main
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchmetrics import R2Score

# implementation
from tools import make_regression_data, RegressionDataset, finite_time_opt_training, drem_opt_training, \
                  standard_training, plot_results
from optimizers import FiniteTimeOptimizer, DREMOptimizer
from torch.optim import Adam, SGD

# graphics
import matplotlib.pyplot as plt

# Data

In [2]:
NUMBER_OF_FEATURES = 10
X_train, X_test, y_train, y_test = make_regression_data(number_samples=1000,
                                                        number_features=NUMBER_OF_FEATURES,
                                                        noise_value=10.5)

print(f'shape of train: {X_train.shape, y_train.shape}\nshape of test: {X_test.shape, y_test.shape}')

train_dataset = RegressionDataset(features=X_train,
                                  labels=y_train)
test_dataset = RegressionDataset(features=X_test,
                                 labels=y_test)
print(f'example of train sample:\n {train_dataset[19]}')


BATCH_SIZE = 10
train_dataloader = DataLoader(dataset=train_dataset,
                              shuffle=True,
                              batch_size=BATCH_SIZE)
test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE)
batch_example_features, batch_example_labels  = next(iter(train_dataloader))
print('shape of batch: features - {} and labels - {}'.format(batch_example_features.shape, batch_example_labels.shape))

shape of train: (torch.Size([800, 10]), torch.Size([800, 1]))
shape of test: (torch.Size([200, 10]), torch.Size([200, 1]))
example of train sample:
 (tensor([-0.7738,  0.7570, -1.7787, -1.2447,  0.8696, -0.9222,  1.4960,  1.3556,
         1.8768,  0.4134]), tensor([11.9213]))
shape of batch: features - torch.Size([10, 10]) and labels - torch.Size([10, 1])


# Model

In [3]:
model = nn.Sequential(nn.Linear(in_features=10, out_features=5, bias=False),
                      nn.ReLU(),
                      nn.Linear(in_features=5, out_features=1, bias=False))

# Hyperparameters

In [4]:
loss_fn = nn.MSELoss()

metric_fn = R2Score()

# we will apply finite-time estimation each 2 batches
N_OF_BATCHES = 2
optimizer = FiniteTimeOptimizer(params=model.parameters(),
                                lr=0.001,
                                n_of_batches=N_OF_BATCHES)

# Train

In [5]:
n_epochs, loss_train, loss_test, score_test = finite_time_opt_training(model=model,
                                                                       optimizer_fn=optimizer,
                                                                       loss_fn=loss_fn,
                                                                       metric_fn=metric_fn,
                                                                       data_train=train_dataloader,
                                                                       data_test=test_dataloader,
                                                                       min_score=0.95,
                                                                       printed=True)

epoch 1: loss = nan and score = nan


In [ ]:
optimizer = Adam(params=model.parameters(), lr=0.001)

n